# Week 2: Hamiltonian Monte Carlo

## Recap

Last week we covered the basics behind MCMC. We talked about why MCMC methods are useful, in particular, when we are trying to work with high-dimensional distributions that are difficult to calculate, but easy to sample. A [new resource](http://www.mcmchandbook.net/HandbookChapter1.pdf) that I found (in the Brilliantly wrong blogpost) nicely ties together a historical background of MCMC methods, as they were developed. The first chapter also introduces the theory of MCMC well, and might help clear up any uncertainties you have regarding how it is different from regular OMC, and goes in depth into some implementation and math of Metropolis-Hastings (in R), as well as Gibbs sampling (which can apparently be thought of as a special case of MH).


## Wiki Summary

Also known as hybrid Monte Carlo, HMC is a MCMC method for sampling a sequence from unknown prob distribution which is difficult to directly sample from, and can be used to approximate the distribution and take the integral of it (to compute expectation). Differs from MH by **reducing corr. of successive sampled states by using Hamiltonian evolution between states**, and targeting states with a higher acceptance criteria than observed prob. dist, which allows it to converge more quickly to absolute pd. 

Take a look at 1.12.10 from [13] (MCMC Handbook) to get a good summary of terminolgy regarding MCMC methods.

### Motivation

Big data, have a lot of (high dim) data, but difficult to fit a complex model to this data. MCMC goes wrong in high dimension for various reasons.

### Why do we want to integrate? (Measure theory aside)
Measure theoretic reason for the overall statistical approach (I don't really get this stuff, but the intuitive argument behind it kinda made sense to me). 

First we want to decide on what kind of statistical questions we want to be asking (or solving) with our data. It can be broken down into optimization (MAP estimation) and integration (calculating expectations). When doing things like MAP estimation, we are essentially solving the following:

$$ f(\hat{\theta}), \hat{\theta} = \arg\max\pi(\theta) $$
where $\pi(\theta)$ is the density function of the distribution. However, the density function is not a fundamental object, they are a computational convenience to express our data. The presenter in [11] posits that it is necessary to characterize the density with the volume, especially in higher dimensions. Calculating expectation on the other hand,

$$ \mathbb{E}[f(\theta)] = \int \mathbf{\text{d}\theta \pi(\theta)}f(\theta)$$

is preferrable, since it takes into consideration $\text{d}\theta$, aka volume.

With increase in number of dimensions, our mass can sit far away from our density, since volume increases. Mass moves away from mode. Look up "concentration of measure."


### Motivation (cont)

We can answer well-posed queries of our data by integrating posterior 

$$\mathbb{E}[f(\theta)] = \int \text{d}\theta f(\theta) \pi(\theta | \mathcal{D})$$

For scalable inference, want to find prob mass (for high d distributions), and then calculate expectations. The reason why this is hard is bc we need to find a efficient way of computing high dimensional integrals. (MCMC, variational bayes, etc. are all trying to do this).

Magic of Monte Carlo algorithm is uncertainty of MC estimator (mean of sampled obs) does not change with number of dimensions (constant scaling w dimension). Now let's cover HMC

## Notes from [4] and [11]

* MCMC occurs in 3 stages:
    * warmup (filtering, learn transition matrix, start from the "typical set", eg. start near mode of data)
    * sampling (fix transition matrix, start rolling out to sample new data)
    * analysis (calculating MC expectations)
* MCMC is designed to mainly work with gaussians. Struggles multi-modality for example. Also struggles in non-convex regions of data distrib with high curvature (which is typically the case in high dims)
* This is because Markov Chain can get stuck, can bias your expectations (when you have pathologies in your data, see Fig 9. from [4] below)
* $ \hat{f} \sim \mathcal{N}(E[f], MCSE^2) $
* Monte Carlo Std Err measures precision of MC estimate, $MCSE^2 = Var(f) / ESS$. Not a problem if samples don't correlate.
* effective sample size (ESS) is roughly number of independent samples generated in chain
* with low autocorrelation, we can do eff bayesian inference, bc independent samples, ESS is high (close to N)
* Hamiltonian MC (as opposed to MCMC) allows you to reduce these autocorr and their impact on the expectation calculation
* Autocorr is calculated by convolving chain with the function you are approximating
* Problem with random walk metropolis/gibbs, transition operators slow exploration and get stuck, RWM an Gibbs explore conditional variance, which means randomness of your initial sample compounds, and get incoherent exploration.
* We need coherent exploration to scale to higher dimensionality. The key is to understand the geometry of your exploration. Use gradients to explore in a coherent way (using integral curves).

<img src="images/2-fig9.png" width=480/>

### How to get integral curves from prob distribution?
* need to add a new term, "momentum," (p)
* Need to define "hamiltonian":
    $$\begin{aligned}H(p,q)
    &= -\log \pi(p,q) \\
    &= -\log \pi(p|q) \pi(q) \\
    &= -\log \pi(p|q) - \log \pi(q)\end{aligned}$$
* $T$ is kinetic energy, first term, quadratic kinetic energies with constant metrics emulate dynamics on a Euclidean manifold. Euclidian Hamiltonian Monte Carlo assumes $T=.5p_ip_j(M^{-1})^{ij}$

## Hamiltonian Monte Carlo (steps, from [8]) 

* Represent states now with $x$ and $p$, where $p$ is momentum. We will be alternating two proposal steps:
    * 1. Randomize $p$ (draw from a gaussian)
    * 2. Simulate Hamiltonian dynamics (defined by Hamiltonian $H(x,p)=E(X) +K(p)$)
       * $E(X)$ (potential energy) corresponds to $-\log\pi(q)$, and $K(p)$ corresponds to $-\log(p|q)$ (or T, kinetic energy)
* Momentum proposal always accepted
* for dynamical proposal, $p$ determins where $x$ goes, and gradient of $E(x)$ determins how $p$ changes, and we use the Metropolis criterion to accept or reject proposal, $$\min(1,\exp\{(H(x,p)-H(x',p')\})$$
* Since H does not change wrt trajectory, dynamical proposals should always be accepted, but sometimes $H$ can reduce due to numerical errors ([9])

## Works Cited
[1] [Hamiltonian Monte Carlo Explained](http://arogozhnikov.github.io/2016/12/19/markov_chain_monte_carlo.html)

[2] [Hybrid Monte-Carlo Sampling (Theano docs)](http://deeplearning.net/tutorial/hmc.html)

[3] [Probabilistic programming in Python
using PyMC3](https://pdfs.semanticscholar.org/8085/b60ce1771647f11ccc4728397275b502f359.pdf)

[4] [A Conceptiual Introduction to Hamiltonian Monte Carlo](https://arxiv.org/pdf/1701.02434.pdf)

[5] [Hamiltonian Monte Carlo Wikipedia](https://en.wikipedia.org/wiki/Hamiltonian_Monte_Carlo)

[6] [MCMC using Hamiltonian Dynamics](https://arxiv.org/pdf/1206.1901.pdf)

[7] [Generalizing Hamiltonian Monte Carlo with Neural Networks](https://openreview.net/pdf?id=B1n8LexRZ)

[8] Information Theory, Inference, and Learning Algorithms §30.1 (Mackay)

[9] Pattern Recognition and Machine Learning §11.5 (Bishop)

[10] [Other interesting papers featuring HMC (Semantic Scholar)](https://www.semanticscholar.org/topic/Hybrid-Monte-Carlo/197800)

[11] [Effecient Bayesian Inference with Hamiltonian Monte Carlo (YouTube)](https://www.youtube.com/watch?v=pHsuIaPbNbY)

[12] [TF Probability (Blogpost)](https://medium.com/tensorflow/introducing-tensorflow-probability-dca4c304e245)

[13] [MCMC Handbook](http://www.mcmchandbook.net/HandbookChapter1.pdf)